In [1]:
import glob, cv2, torch
import torch.nn as nn
from PIL import Image

import pandas as pd
import numpy as np
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
torch.cuda.is_available()

True

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [3]:
dogs = glob.glob("cat_and_dog/Dog/*")
cats = glob.glob("cat_and_dog/Cat/*")

In [4]:
df = pd.concat([
    pd.DataFrame({"label":1,"path":dogs}),
    pd.DataFrame({"label":0,"path":cats})
], ignore_index = True).sample(frac=1, random_state=42).reset_index(drop=True)

In [5]:
class BinaryImageDataset(torch.utils.data.Dataset):
    def __init__(self, df, img_size=(224,224), transform=None):
        self.df = df.reset_index(drop=True)
        self.img_size = img_size
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'path']
        label = self.df.loc[idx, 'label']

        # img = cv2.imread(img_path)  # BGR format
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # RGB로 변환
        # img = cv2.resize(img, self.img_size)       # (width, height)
        # numpy -> tensor, C,H,W, 0~1
        # img = torch.from_numpy(img).permute(2,0,1).float() / 255.0

        # Normalize (ImageNet 기준)
        # mean = torch.tensor([0.485,0.456,0.406]).view(3,1,1)
        # std  = torch.tensor([0.229,0.224,0.225]).view(3,1,1)
        # img = (img - mean)/std

        
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float32)

In [6]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # ImageNet mean
                         std=[0.229, 0.224, 0.225])    # ImageNet std
])

In [7]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_dataset = BinaryImageDataset(train_df, transform=transform)
val_dataset = BinaryImageDataset(val_df, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [15]:
import pytorch_lightning as pl
class SimpleMobileNet(pl.LightningModule):
    def __init__(self, lr=2e-5, scheduler_patience = 3):
        super().__init__()
        self.lr = lr
        backbone  = models.mobilenet_v2(
            weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        backbone.classifier = nn.Identity()
        for p in backbone.features.parameters(): 
            p.requires_grad = False
        self.backbone = backbone
        
        self.criterion = nn.BCEWithLogitsLoss()
        self.head = nn.Sequential(
            nn.Linear(1280, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()  
        )
    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

    def training_step(self, batch, batch_idx):
        x,y = batch
        y = y.unsqueeze(1)
        loss = self.criterion(self(x), y)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# 체크포인트 콜백: val_loss 기준으로 최적 모델 저장
checkpoint_cb = ModelCheckpoint(
    monitor='val_loss',      # 모니터링할 metric
    mode='min',              # val_loss가 낮을수록 좋음
    save_top_k=1,            # 가장 좋은 1개만 저장
    filename='best_model'    # 저장될 파일 이름
)

# 얼리스탑 콜백: val_loss 개선 없으면 조기 종료
earlystop_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,              # 5 epoch 동안 개선 없으면 종료
    mode='min'
)

In [ ]:
from pytorch_lightning import Trainer
model = SimpleMobileNet()
loss_callback = LossHistoryCallback()

trainer = Trainer(
    max_epochs=10,
    accelerator='auto', # GPU 자동 감지
    devices=1,# GPU 하나 사용 (없으면 CPU)
    callbacks=[checkpoint_cb, earlystop_cb],
)
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | backbone  | MobileNetV2       | 2.2 M 
1 | criterion | BCEWithLogitsLoss | 0     
2 | head      | Sequential        | 328 K 
------------------------------------------------
328 K     Trainable params
2.2 M     Non-trainable params
2.6 M     Total params
10.208    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# batch 단위 loss → epoch 평균
train_losses = np.array(loss_callback.train_losses).reshape(trainer.max_epochs, -1).mean(axis=1)
val_losses = np.array(loss_callback.val_losses).reshape(trainer.max_epochs, -1).mean(axis=1)

plt.figure(figsize=(8,5))
plt.plot(range(1, trainer.max_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, trainer.max_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train / Validation Loss Curve')
plt.legend()
plt.show()


In [ ]:
# ------------------------
class SimpleBinaryModel(nn.Module):
    def __init__(self, input_dim=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # BCEWithLogitsLoss 사용 → Sigmoid 제거
        )

    def forward(self, x):
        return self.model(x)

model = SimpleBinaryModel()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# ------------------------
# 3. 손실 함수 & 옵티마이저
# ------------------------
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ------------------------
# 4. 학습 루프
# ------------------------
num_epochs = 10

for epoch in range(num_epochs):
    # --- train ---
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device).unsqueeze(1)  # [B] -> [B,1]

        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)

    train_loss /= len(train_loader.dataset)

    # --- validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device).unsqueeze(1)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            val_loss += loss.item() * x.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")